# Mobile Get Input Notebook - Phase 5: P.1812 Loss Calculation

**Phase 5**: Process Phase 4 CSV profiles through ITU-R P.1812-6 model and calculate radio propagation loss.

This notebook assumes Phase 0 setup has been completed. It will:
- Load previously generated CSV profiles from Phase 4
- Add transmitter point (distance=0) required by P.1812
- Process each profile using P.1812 bt_loss() function
- Calculate basic transmission loss (Lb) and electric field strength (Ep)
- Export results to structured DataFrame and CSV file

## Prerequisites
- Phase 0: Setup (defines CONFIG, project_root, data directories)
- Phase 1-4: Must be completed to generate CSV profiles

## Output
- CSV file: `data/output/spreadsheets/p1812_results.csv`
- Summary statistics and visualizations

## 1. Setup from Phase 0

Run Phase 0 first to initialize environment

In [ ]:
%run phase0_setup.ipynb

## 2. Additional Imports for Phase 5

In [ ]:
import ast
import csv
import time
from typing import List, Tuple

# P.1812-6 propagation model
import Py1812.P1812
import matplotlib.pyplot as plt

print("✓ Phase 5 imports successful")

## 3. Define Output Paths

Using data directories from Phase 0 setup

In [ ]:
# Input: CSV profiles from Phase 4
# (project_root and profiles_dir already defined from Phase 0)

# Output: Results directory
output_dir = project_root / 'data' / 'output' / 'spreadsheets'
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Input profiles dir: {profiles_dir}")
print(f"Output results dir: {output_dir}")

# Verify Phase 4 profiles exist
csv_files = list(profiles_dir.glob("*.csv"))
print(f"\nFound {len(csv_files)} CSV profile file(s)")
if csv_files:
    for f in csv_files:
        print(f"  - {f.name}")

## 4. Load CSV Profiles

Load all semicolon-delimited profiles from Phase 4 output

In [ ]:
def load_profiles(profiles_dir: Path) -> List[List[str]]:
    """
    Load all CSV profiles from directory.
    Skip header rows.
    """
    profiles = []
    
    csv_files = list(profiles_dir.glob("*.csv"))
    print(f"Loading {len(csv_files)} profile file(s)...")
    
    for file in csv_files:
        print(f"  Loading: {file.name}")
        with file.open(newline="", encoding="utf-8") as f:
            # Skip header (first line), read remaining rows
            rows = list(csv.reader(f, delimiter=";"))[1:]
            profiles.extend(rows)
            print(f"    -> {len(rows)} profiles loaded")
    
    print(f"\nTotal profiles: {len(profiles)}")
    return profiles

# Load all profiles
profiles = load_profiles(profiles_dir)

if not profiles:
    print("\n⚠ WARNING: No profiles found!")
    print("Make sure Phase 4 has been executed to generate CSV profiles.")

## 5. Parse Profile Parameters with Transmitter Point Fix

Convert CSV strings to numpy arrays. **Important**: P.1812 requires d[0]=0 (transmitter point). If Phase 4 profiles start at 0.03 km, we prepend the TX point.

In [ ]:
def process_loss_parameters(profile: List[str]) -> Tuple:
    """
    Parse CSV row into P.1812 bt_loss() parameters.
    
    CSV columns (first 14 used, skip azimuth at index 14):
    0: f (frequency)
    1: p (time percentage)
    2: d (distance array)
    3: h (height array)
    4: R (resistance array)
    5: Ct (land cover category array)
    6: zone (zone ID array)
    7: htg (TX antenna height)
    8: hrg (RX antenna height)
    9: pol (polarization)
    10: phi_t (TX latitude)
    11: phi_r (RX latitude)
    12: lam_t (TX longitude)
    13: lam_r (RX longitude)
    14: azimuth (skip - not needed for P.1812)
    
    IMPORTANT: P.1812 requires d[0] = 0 (transmitter point).
    If Phase 4 profiles start at 0.03 km, we prepend a TX point.
    """
    # Parse first 14 columns
    parameters = [ast.literal_eval(parameter) for parameter in profile[0:14]]
    
    # Extract arrays
    d = np.array([float(v) for v in parameters[2]])
    h = np.array([float(v) for v in parameters[3]])
    R = np.array([float(v) for v in parameters[4]])
    Ct = np.array([int(v) for v in parameters[5]])
    zone = np.array([int(v) for v in parameters[6]])
    
    # P.1812 requires first distance point to be 0 (at transmitter)
    # If first point is not 0, prepend TX point
    if d[0] != 0:
        # Use first receiver point's properties for TX point
        d = np.concatenate([[0], d])
        h = np.concatenate([[h[0]], h])      # TX height same as first RX
        R = np.concatenate([[R[0]], R])      # TX resistance same as first point
        Ct = np.concatenate([[Ct[0]], Ct])   # TX land cover same as first point
        zone = np.concatenate([[zone[0]], zone])  # TX zone same as first point
    
    return (
        float(parameters[0]),                                  # f: frequency (GHz)
        float(parameters[1]),                                  # p: time percentage (%)
        d,                                                      # d: distances (km)
        h,                                                      # h: heights (m)
        R,                                                      # R: resistances (ohms)
        Ct,                                                     # Ct: land cover categories
        zone,                                                   # zone: zone IDs
        float(parameters[7]),                                  # htg: TX antenna height (m)
        float(parameters[8]),                                  # hrg: RX antenna height (m)
        int(parameters[9]),                                    # pol: polarization (1=horiz, 2=vert)
        float(parameters[10]),                                 # phi_t: TX latitude
        float(parameters[11]),                                 # phi_r: RX latitude
        float(parameters[12]),                                 # lam_t: TX longitude
        float(parameters[13]),                                 # lam_r: RX longitude
    )

# Test on first profile
if profiles:
    params = process_loss_parameters(profiles[0])
    print(f"Example profile parameters (profile 1):")
    print(f"  Frequency: {params[0]} GHz")
    print(f"  Time %: {params[1]}%")
    print(f"  Distance points: {len(params[2])} (now includes TX point at d[0]=0)")
    print(f"  First 5 distances: {params[2][:5]} km")
    print(f"  TX location: lat={params[10]:.4f}, lon={params[12]:.4f}")
    print(f"  RX location: lat={params[11]:.4f}, lon={params[13]:.4f}")
    print(f"  Max distance: {params[2][-1]:.2f} km")

## 6. Execute P.1812 Loss Calculation

In [ ]:
def calculate_p1812_loss(parameters: Tuple) -> Tuple[float, float]:
    """
    Call P.1812 bt_loss() function.
    
    Returns (Lb, Ep) where:
    - Lb: Basic transmission loss (dB)
    - Ep: Electric field strength (dBμV/m)
    """
    try:
        Lb, Ep = Py1812.P1812.bt_loss(*parameters)
        return float(Lb), float(Ep)
    except Exception as e:
        print(f"  ⚠ Error calculating loss: {e}")
        return np.nan, np.nan

# Test on first profile
if profiles:
    print("Testing P.1812 calculation on first profile...")
    params = process_loss_parameters(profiles[0])
    start = time.perf_counter()
    Lb, Ep = calculate_p1812_loss(params)
    elapsed = time.perf_counter() - start
    
    if not np.isnan(Lb):
        print(f"\n✓ Calculation successful:")
        print(f"  Basic transmission loss (Lb): {Lb:.2f} dB")
        print(f"  Electric field strength (Ep): {Ep:.2f} dBμV/m")
    else:
        print(f"\n✗ Calculation failed (NaN result)")
    print(f"  Calculation time: {elapsed:.4f} seconds")

## 7. Process All Profiles and Store Results

In [ ]:
def process_all_profiles(profiles: List[List[str]]) -> pd.DataFrame:
    """
    Process all profiles through P.1812 and collect results.
    
    Returns DataFrame with:
    - profile_id: Profile index
    - frequency_ghz, time_percentage, polarization: P.1812 parameters
    - tx_lat, tx_lon, rx_lat, rx_lon: Location data
    - antenna_height_tx_m, antenna_height_rx_m: Antenna heights
    - max_distance_km, num_distance_points: Profile geometry
    - Lb_dB: Basic transmission loss
    - Ep_dBuV_m: Electric field strength
    """
    results = []
    failed_profiles = 0
    
    start_total = time.perf_counter()
    
    for idx, profile in enumerate(profiles):
        try:
            params = process_loss_parameters(profile)
            Lb, Ep = calculate_p1812_loss(params)
            
            # Extract metadata from parameters
            f, p, d, h, R, Ct, zone, htg, hrg, pol, phi_t, phi_r, lam_t, lam_r = params
            
            results.append({
                'profile_id': idx + 1,
                'frequency_ghz': f,
                'time_percentage': p,
                'polarization': pol,
                'tx_lat': phi_t,
                'tx_lon': lam_t,
                'rx_lat': phi_r,
                'rx_lon': lam_r,
                'antenna_height_tx_m': htg,
                'antenna_height_rx_m': hrg,
                'max_distance_km': float(d[-1]),
                'num_distance_points': len(d),
                'Lb_dB': Lb,
                'Ep_dBuV_m': Ep,
            })
            
            if (idx + 1) % 10 == 0:
                print(f"  Processed {idx + 1}/{len(profiles)} profiles...")
        
        except Exception as e:
            print(f"  ✗ Failed on profile {idx + 1}: {e}")
            failed_profiles += 1
            continue
    
    elapsed_total = time.perf_counter() - start_total
    
    print(f"\n✓ Processing complete")
    print(f"  Successful: {len(results)}/{len(profiles)}")
    print(f"  Failed: {failed_profiles}")
    print(f"  Total time: {elapsed_total:.2f} seconds")
    if len(profiles) > 0:
        print(f"  Avg per profile: {elapsed_total/len(profiles):.4f} seconds")
    
    return pd.DataFrame(results)

# Process all profiles
if profiles:
    print(f"Processing {len(profiles)} profiles...\n")
    results_df = process_all_profiles(profiles)
else:
    print("No profiles to process. Skipping Phase 5 calculations.")
    results_df = pd.DataFrame()

## 8. Examine Results

In [ ]:
if not results_df.empty:
    print(f"Results shape: {results_df.shape}")
    print(f"\nFirst 5 rows:")
    print(results_df.head())
    print(f"\nColumn names:")
    print(list(results_df.columns))
    print(f"\nStatistics:")
    print(results_df[['Lb_dB', 'Ep_dBuV_m', 'max_distance_km']].describe())
else:
    print("No results to display.")

## 9. Export Results to CSV

In [ ]:
if not results_df.empty:
    # Export to CSV
    csv_path = output_dir / 'p1812_results.csv'
    results_df.to_csv(csv_path, index=False)
    
    print(f"✓ Exported results to:")
    print(f"  {csv_path}")
    print(f"  File size: {csv_path.stat().st_size / 1024:.1f} KB")
    print(f"  Rows: {len(results_df)}")
else:
    print("No results to export.")

## 10. Visualizations

In [ ]:
if not results_df.empty:
    # Plot loss distributions
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Basic transmission loss distribution
    ax1.hist(results_df['Lb_dB'].dropna(), bins=30, edgecolor='black', alpha=0.7, color='steelblue')
    ax1.set_xlabel('Basic Transmission Loss (dB)', fontsize=11)
    ax1.set_ylabel('Frequency', fontsize=11)
    ax1.set_title('Distribution of Lb Values', fontsize=12, fontweight='bold')
    ax1.grid(alpha=0.3)
    
    # Electric field strength distribution
    ax2.hist(results_df['Ep_dBuV_m'].dropna(), bins=30, edgecolor='black', alpha=0.7, color='darkorange')
    ax2.set_xlabel('Electric Field Strength (dBμV/m)', fontsize=11)
    ax2.set_ylabel('Frequency', fontsize=11)
    ax2.set_title('Distribution of Ep Values', fontsize=12, fontweight='bold')
    ax2.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Scatter: loss vs distance
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.scatter(results_df['max_distance_km'], results_df['Lb_dB'], alpha=0.6, s=50, color='darkred')
    ax.set_xlabel('Maximum Distance (km)', fontsize=11)
    ax.set_ylabel('Basic Transmission Loss (dB)', fontsize=11)
    ax.set_title('Loss vs Distance', fontsize=12, fontweight='bold')
    ax.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("No results to visualize.")

## 11. Summary

In [ ]:
print("\n" + "="*70)
print("PHASE 5: P.1812 LOSS CALCULATION - SUMMARY")
print("="*70)

print(f"\nInput:")
print(f"  Location: {profiles_dir}")
print(f"  Total profiles: {len(profiles)}")

if not results_df.empty:
    print(f"\nResults:")
    print(f"  Successful: {len(results_df)}")
    print(f"  Failed: {len(profiles) - len(results_df)}")
    
    valid_lb = results_df['Lb_dB'].dropna()
    valid_ep = results_df['Ep_dBuV_m'].dropna()
    
    if len(valid_lb) > 0:
        print(f"\nBasic Transmission Loss (Lb):")
        print(f"  Min: {valid_lb.min():.2f} dB")
        print(f"  Max: {valid_lb.max():.2f} dB")
        print(f"  Mean: {valid_lb.mean():.2f} dB")
        print(f"  Std: {valid_lb.std():.2f} dB")
    
    if len(valid_ep) > 0:
        print(f"\nElectric Field Strength (Ep):")
        print(f"  Min: {valid_ep.min():.2f} dBμV/m")
        print(f"  Max: {valid_ep.max():.2f} dBμV/m")
        print(f"  Mean: {valid_ep.mean():.2f} dBμV/m")
        print(f"  Std: {valid_ep.std():.2f} dBμV/m")
    
    csv_path = output_dir / 'p1812_results.csv'
    print(f"\nOutput:")
    print(f"  Location: {csv_path}")
    print(f"  Rows: {len(results_df)}")
    print(f"  Columns: {len(results_df.columns)}")
else:
    print(f"\nNo results generated. Ensure Phase 4 profiles exist.")

print("="*70 + "\n")